<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 2: Practical - LDA & QDA <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Arshia Soltani Moakhar
    
____

### Full Name : Parsa Sharifi
### Student Number : 99101762
___

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Exploration (20 points)

In [ ]:
!wget https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv

--2023-04-14 14:50:30--  https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv [following]
--2023-04-14 14:50:31--  https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89401 (87K) [text/plain]
Saving to: ‘dataset-train.csv.3’

dataset-train.csv.3 100%[===================>]  87.31K  --.-KB/s    in 0.006s  

2023-04-14 14:50:31 (14.9 MB/s) - ‘dataset-train.csv



Load the dataset as a dataframe

In [ ]:
df = pd.read_csv("dataset-train.csv")
print(df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

                           Ad Topic Line            City     Country  \
0     Cloned 5thgeneration orchestration     Wrightburgh     Tunisia   
1     Monitored national standardization       West Jodi       Nauru   
2       Organic bottom-line service-desk        Davidton  San Marino   
3  Triple-buffered reciprocal time-frame  West Terrifurt       Italy   
4          Robust logistical utilization    South Manuel     Iceland   

             Timestamp  Clicked on Ad  Gender  
0  2016-03-27 00:53:11              0  female  
1  2016-04-04 01:39:02              0    mal

Get the minimum "Area Income" for all Italians

In [ ]:
italian_df = df[df['Country'] == 'Italy']
min_income = italian_df['Area Income'].min()

In [ ]:
print(min_income)

54286.1


Make a new dataframe called "sample_df" by sampling 100 rows from the previously made dataframe

In [ ]:
sample_df = df.sample(n=100, random_state=42)

In [ ]:
print(sample_df.head())

     Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
696                     39.19   54     52581.16                173.05   
667                     79.61   31     58342.63                235.97   
63                      86.06   32     61601.05                178.92   
533                     91.43   39     46964.11                209.91   
66                      63.89   40     51317.33                105.22   

                        Ad Topic Line             City           Country  \
696  Stand-alone empowering benchmark         Leahside         Guatemala   
667  Customizable value-added project         Luisfurt            Monaco   
63       Focused upward-trending core  North Leonmouth          Bulgaria   
533          Organic motivating model  West Carmenfurt  French Polynesia   
66      Synchronized user-facing core    Jensenborough            Rwanda   

               Timestamp  Clicked on Ad  Gender  
696  2016-01-23 21:15:57              1  female  
667 

Compare all men and women aged less than 30 years in terms of their average "Daily Internet Usage". Your result should contain the average daily usage for each age and gender, seperately.

In [ ]:
young_df = df[df['Age'] < 30]
result = young_df.groupby(['Gender', 'Age'])['Daily Internet Usage'].mean()

In [ ]:
print(result)

Gender  Age
female  19     184.940000
        20     202.160000
        21     186.980000
        22     200.070000
        23     187.105714
        24     178.693333
        25     193.618571
        26     192.741875
        27     189.157692
        28     205.873333
        29     191.191200
male    19     189.523333
        20     198.982500
        21     212.815000
        22     188.837500
        23     184.615000
        24     200.055556
        25     206.527857
        26     203.916250
        27     205.916154
        28     196.416842
        29     202.005714
Name: Daily Internet Usage, dtype: float64


# Preprocessing (30 points)

Remove the following columns: "Ad Topic Line", "Timestamp"

In [ ]:
df = df.drop(['Ad Topic Line', 'Timestamp'], axis=1)

In [ ]:
print(df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

             City     Country  Clicked on Ad  Gender  
0     Wrightburgh     Tunisia              0  female  
1       West Jodi       Nauru              0    male  
2        Davidton  San Marino              0  female  
3  West Terrifurt       Italy              0    male  
4    South Manuel     Iceland              0  female  


Convert the "City" and "Country" columns to their one-hot encoded versions.
For each set of one-hot encoded columns, remove one of them to make the columns not correlated. Finally, remove the original columns named "City" and "Country".

In [ ]:
city_dummies = pd.get_dummies(df['City'], prefix='City')
country_dummies = pd.get_dummies(df['Country'], prefix='Country')

df = pd.concat([df, city_dummies, country_dummies], axis=1)

print(df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

             City     Country  Clicked on Ad  Gender  City_Adamsbury  \
0     Wrightburgh     Tunisia              0  female               0   
1       West Jodi       Nauru              0    male               0   
2        Davidton  San Marino              0  female               0   
3  West Terrifurt       Italy              0    male               0   
4    South Manuel     Iceland              0  female               0   

   City_Alanview  ...  Country_Uruguay  Country_Uzbekistan  Country_Vanuatu  \
0              0  ...                0                   0   

In [ ]:
df1 = df.loc[:,~df.columns.duplicated()]

In [ ]:
df = df.drop(['Country_Yemen'], axis=1)

In [ ]:
df = df.drop(['City_Adamsbury'], axis=1)

In [ ]:
df.drop(["City", "Country"], axis=1, inplace=True)

In [ ]:
print(df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

   Clicked on Ad  Gender  City_Alanview  City_Alexanderfurt  \
0              0  female              0                   0   
1              0    male              0                   0   
2              0  female              0                   0   
3              0    male              0                   0   
4              0  female              0                   0   

   City_Alexanderview  City_Alexisland  ...  Country_United States of America  \
0                   0                0  ...                                 0   
1                   0           

In [ ]:
num_columns = df.shape[1]
print(num_columns)

1017


Convert "Gender" column to a new binary column (having only 0 and 1 values). Then remove the original column.

In [ ]:
df['Binary_Gender'] = df['Gender'].map({'male': 1, 'female': 0})

In [ ]:
df = df.drop(['Gender'], axis=1)

In [ ]:
print(df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

   Clicked on Ad  City_Alanview  City_Alexanderfurt  City_Alexanderview  \
0              0              0                   0                   0   
1              0              0                   0                   0   
2              0              0                   0                   0   
3              0              0                   0                   0   
4              0              0                   0                   0   

   City_Alexisland  City_Aliciatown  ...  Country_Uruguay  Country_Uzbekistan  \
0                0                0  ... 

In [ ]:
num_columns = df.shape[1]
print(num_columns)

1017


By dropping the "Clicked on Ad" column, make a new dataframe called "features_df"

In [ ]:
features_df = df.drop(['Clicked on Ad'], axis=1)

In [ ]:
print(features_df.head())

   Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

   City_Alanview  City_Alexanderfurt  City_Alexanderview  City_Alexisland  \
0              0                   0                   0                0   
1              0                   0                   0                0   
2              0                   0                   0                0   
3              0                   0                   0                0   
4              0                   0                   0                0   

   City_Aliciatown  City_Alvaradoport  ...  Country_Uruguay  \
0                0                  0  ...     

Make a new dataframe called "target_df" containing only the "Clicked on Ad" column

In [ ]:
target_df = df.loc[:, 'Clicked on Ad']

Make two numpy arrays called "x" and "y" from "features_df" and "target_df" dataframes, respectively.

In [ ]:
x = features_df.values
y = target_df.values

In [ ]:
print(x)
print(y)

[[6.895000e+01 3.500000e+01 6.183390e+04 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [8.023000e+01 3.100000e+01 6.844185e+04 ... 0.000000e+00 0.000000e+00
  1.000000e+00]
 [6.947000e+01 2.600000e+01 5.978594e+04 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 ...
 [7.705000e+01 3.400000e+01 6.575636e+04 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [6.683000e+01 4.600000e+01 7.787175e+04 ... 0.000000e+00 0.000000e+00
  1.000000e+00]
 [7.620000e+01 2.400000e+01 4.725859e+04 ... 0.000000e+00 0.000000e+00
  1.000000e+00]]
[0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 0 1
 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1
 1 0 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 1
 1 0 1 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 1 1 1
 1 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 1 0 0
 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1 0 0
 1 1 0 1 1 1 0 0 0 1 1

# Training Models (40 points)

Split the dataset (`x` and `y`) into "training" and "testing" parts containing 80% and 20% of the data respectively.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Train both LDA and QDA models on the data using `sklearn`

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)

LinearDiscriminantAnalysis()

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

Train LDA and QDA using your own code!

Linear Discriminant Analysis

In [ ]:
class LinearDiscriminantAnalysis:

    def __init__(self):
        self.linear_discriminants = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        class_labels = np.unique(y)

        total_mean = np.mean(X, axis=0)
        SW = np.zeros((n_features, n_features))
        SB = np.zeros((n_features, n_features))

        for c in class_labels:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            SW += (X_c - mean_c).T.dot((X_c - mean_c))
            n_c = X_c.shape[0]
            difference_of_mean = (mean_c - total_mean).reshape(n_features, 1)
            SB += n_c * (difference_of_mean).dot(difference_of_mean.T)

        SW += np.eye(n_features) * 1e-8  # regularization with a small positive constant
        SW_inv = np.linalg.inv(SW)
        A = SW_inv.dot(SB)

        eigenvalues, eigenvectors = np.linalg.eig(A)

        eigenvectors = eigenvectors.T
        idxs = np.argsort(abs(eigenvalues))[::-1]
        eigenvectors = eigenvectors[idxs]

        self.linear_discriminants = eigenvectors[0:class_labels.size - 1]
        
        return self

    def predict(self, X):
        # found the treshold by gettin mean of means of 2 classes!! =)
        threshold = .8
        y_pred = np.where(np.dot(X, self.linear_discriminants.T) > threshold, 1, 0)
        return y_pred

Quadratic Discriminant Analysis (Optional) (15 bonus points)

In [ ]:
class QuadraticDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        pass

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        self.class_labels = np.unique(y)
        self.priors = np.zeros(len(self.class_labels))


        #self.covariance_matrix = np.zeros((n_features, n_features))
        self.covariance_matrices = [np.zeros((n_features, n_features)) for _ in np.unique(y)]
        self.mean_of_classes = np.zeros((len(self.class_labels), n_features))


        for i, c in enumerate(self.class_labels):
            X_c = X[y == c]
            self.priors[i] = float(X_c.shape[0]) / float(n_samples)
            self.mean_of_classes[i] = np.mean(X_c, axis=0)

            #self.covariance_matrix+= np.cov(X_c.T)
            self.covariance_matrices[i] = np.cov(X_c.T)

        return self

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape


        discriminant_functions = np.zeros((self.class_labels.shape[0], n_samples))
        for i in range(self.class_labels.shape[0]):
            covariance_inverse = np.linalg.pinv(self.covariance_matrices[i])
            mean_vector = self.mean_of_classes[i]
            priors = self.priors[i]
            discriminant_functions[i] = (-0.5 * np.sum(np.dot((X - mean_vector), covariance_inverse) * (X - mean_vector), axis=1)
                                         - 0.5 * np.log(np.linalg.det(self.covariance_matrices[i]))
                                         + np.log(priors))


        #i also saw this kind of implimentation which is below, and it works better than what is said in note6! for using the other method change the place of # for covariance matixes and code below

        #discriminant_functions = np.zeros((n_samples, len(self.class_labels)))
#
        #for i, c in enumerate(self.class_labels):
        #    discriminant_functions[:, i] = -0.5 * np.sum((X - self.mean_of_classes[i]).dot(np.linalg.pinv(self.covariance_matrix)) * (X -  self.mean_of_classes[i]), axis=1) + np.log(self.priors[i])
#
        y_pred = self.class_labels[np.argmax(discriminant_functions, axis=1)]

        return y_pred

In [ ]:
# Train your own models

In [ ]:
my_lda = LinearDiscriminantAnalysis()

my_lda.fit(x_train, y_train)

In [ ]:
my_qda = QuadraticDiscriminantAnalysis()

my_qda.fit(x_train, y_train)

# Evaluation (10 points)

Using all 4 models, predict the labels for all samples belonging to the "testing" part

In [ ]:
y_pred_sklda = lda.predict(x_test)
y_pred_skqda = qda.predict(x_test) 
 

In [ ]:
y_pred_lda = my_lda.predict(x_test)


In [ ]:
y_pred_qda = my_qda.predict(x_test)

In [ ]:
y_pred_lda

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

Calculate Accuracy, Recall, and F1-Scores

In [ ]:

from sklearn.metrics import accuracy_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred_sklda)
recall = recall_score(y_test, y_pred_sklda)
f1 = f1_score(y_test, y_pred_sklda)

print(accuracy)
print(recall)
print(f1)

0.9
0.8658536585365854
0.8987341772151899


In [ ]:

from sklearn.metrics import accuracy_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred_skqda)
recall = recall_score(y_test, y_pred_skqda)
f1 = f1_score(y_test, y_pred_skqda)

print(accuracy)
print(recall)
print(f1)

0.9
0.9634146341463414
0.9080459770114943


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred_lda)
recall = recall_score(y_test, y_pred_lda)
f1 = f1_score(y_test, y_pred_lda)

print(accuracy)
print(recall)
print(f1)

0.4875
0.0
0.0


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred_qda)
recall = recall_score(y_test, y_pred_qda)
f1 = f1_score(y_test, y_pred_qda)

print(accuracy)
print(recall)
print(f1)

0.9375
0.9024390243902439
0.9367088607594938


Compute the ROC AUC Score using `sklearn` (for both LDA and QDA models trained using this library)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, y_pred_sklda)
print(roc_auc)

0.9008755472170106


In [ ]:
roc_auc = roc_auc_score(y_test, y_pred_skqda)
print(roc_auc)

0.8983739837398375


# Kaggle Competition

Link: https://www.kaggle.com/t/988b0aa2db2446fd85cff571d69c6386